In [1]:
import os
import glob
import torch
import torch.nn as nn
import torchaudio
from fairseq.models.wav2vec import Wav2VecModel
from sklearn.preprocessing import OneHotEncoder

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '4'

In [3]:
torch.cuda.current_device()

0

In [4]:
torch.cuda.device_count()

1

In [5]:
# dev = torch.device('cpu')

In [5]:
dev = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [7]:
# Load Wav2VecModel

In [6]:
cp = torch.load('/home/karmanya/wav2vec_large.pt');
model = Wav2VecModel.build_model(cp['args'], task=None);
model.load_state_dict(cp['model']);
model.to(dev);

Wav2VecModel(
  (feature_extractor): ConvFeatureExtractionModel(
    (conv_layers): ModuleList(
      (0): Sequential(
        (0): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (1): Dropout(p=0.0, inplace=False)
        (2): Fp32GroupNorm(1, 512, eps=1e-05, affine=True)
        (3): ReLU()
      )
      (1): Sequential(
        (0): Conv1d(512, 512, kernel_size=(8,), stride=(4,), bias=False)
        (1): Dropout(p=0.0, inplace=False)
        (2): Fp32GroupNorm(1, 512, eps=1e-05, affine=True)
        (3): ReLU()
      )
      (2): Sequential(
        (0): Conv1d(512, 512, kernel_size=(4,), stride=(2,), bias=False)
        (1): Dropout(p=0.0, inplace=False)
        (2): Fp32GroupNorm(1, 512, eps=1e-05, affine=True)
        (3): ReLU()
      )
      (3): Sequential(
        (0): Conv1d(512, 512, kernel_size=(4,), stride=(2,), bias=False)
        (1): Dropout(p=0.0, inplace=False)
        (2): Fp32GroupNorm(1, 512, eps=1e-05, affine=True)
        (3): ReLU()
      )
 

In [20]:
F = nn.functional
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x):
        return self.func(x)

In [ ]:
shrink = nn.Sequential(
    nn.AdaptiveMaxPool2d((512,1)),
    Lambda(lambda x: x.view(-1, 512)),
    nn.Linear(512,3),
    nn.Softmax()
).to(dev)

In [7]:
data_files_path = '/media/nas_mount/Sarthak/ijcai_acl/prompt_wise_16k_audios/prompt3/'
data_files_list = glob.glob(data_files_path+'*.wav')

In [8]:
labels_path = '/media/nas_mount/Sarthak/ques_wise_models/p3q1/labels.csv'
test_path = '/media/nas_mount/Sarthak/ques_wise_models/p3q1/test.csv'

In [9]:
import pandas as pd
df = pd.read_csv(labels_path)

In [10]:
test_frame = pd.read_csv(test_path)

In [11]:
test_frame.groupby('label').count()

,name
label,
0,64
1,327
2,1198


In [ ]:
df.groupby('label').count()

In [14]:
enc = OneHotEncoder(sparse=False)
training_labels = enc.fit_transform(df['label'].to_numpy().reshape(-1, 1))
test_labels = enc.transform(test_frame['label'].to_numpy().reshape(-1,1))

/home/karmanya/.conda/envs/asr/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [15]:
# waveform, sr = torchaudio.load(data_files_list[3])
# test(waveform)

In [16]:
# torchaudio.load(os.path.join(data_files_path, (df.iloc[4]['name'].split('.jpeg')[0]+'.wav')))[0]

In [88]:
from torch.utils.data import DataLoader, Dataset

class AudioDataset(Dataset):
    def __init__(self, df, audio_dir, labels, file_col, model):
        '''
        Passed Dataset a dataframe of the filenames to train/validate on
        Pass one hot encoded numpy array for labels
        Pass a string for the directory of audio files
        '''
        self.items = df
        self.model = model
        self.items['path'] = self.items[file_col].apply(lambda x: x.split('.jpeg')[0]) # Label files are *.jpeg
        self.items['path'] = self.items['path'].apply(lambda x: os.path.join(audio_dir, f'{x}.wav'))
        self.labels = labels
        self.audio_dir = audio_dir
#         self.model.eval()
    def __len__(self):
        return len(self.items)
    
    def __getitem__(self, idx):
        '''
        Load the audio waveform of idx from dataframe
        returns a touple of torch tensor and label
        '''
        file = self.items['path'].iloc[idx]
        label = self.labels[idx]
        audio, _ = torchaudio.load(file)
        samples = audio.shape[1]
        # Trim and pad to 60 seconds
        if samples < 60*16000:
            p1d = (60*16000 - samples, 0)
            audio = F.pad(audio, p1d, "constant", 0)
        elif samples > 60*16000 :
            audio = torch.narrow(audio, 1, 0, 60*16000)
        return audio, torch.tensor(label).type('torch.FloatTensor')
#         with torch.no_grad():
#             z = self.model.feature_extractor(audio)
#             c = self.model.feature_aggregator(z)
            
#         return c.to(dev), torch.tensor(label).type('torch.FloatTensor').to(dev)

In [89]:
train_dataset = AudioDataset(df, data_files_path, training_labels, 'name', model)
# test_dataset = AudioDataset(test_frame, data_files_path, test_labels,'name', model )

In [90]:
train_dataset[0][0].shape

torch.Size([1, 960000])

In [58]:
samples = [sample[0].shape[1] for sample in train_dataset]

In [96]:
train_dataset[0][0].view(-1, 30*16000)[0].shape

torch.Size([480000])

In [100]:
torch.unsqueeze(train_dataset[0][0].view(-1, 30*16000)[0], 0)

tensor([[-3.0518e-05,  0.0000e+00,  0.0000e+00,  ...,  2.7466e-04,
          3.3569e-04,  2.4414e-04]])

In [101]:
model.feature_extractor(model.feature_aggregator(torch.unsqueeze(train_dataset[0][0].view(-1, 30*16000)[0], 0)))

NotImplementedError: Only 3D, 4D, 5D padding with non-constant padding are supported for now

In [86]:
torch.narrow?

Docstring:
narrow(input, dim, start, length) -> Tensor

Returns a new tensor that is a narrowed version of :attr:`input` tensor. The
dimension :attr:`dim` is input from :attr:`start` to :attr:`start + length`. The
returned tensor and :attr:`input` tensor share the same underlying storage.

Args:
    input (Tensor): the tensor to narrow
    dim (int): the dimension along which to narrow
    start (int): the starting dimension
    length (int): the distance to the ending dimension

Example::

    >>> x = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
    >>> torch.narrow(x, 0, 0, 2)
    tensor([[ 1,  2,  3],
            [ 4,  5,  6]])
    >>> torch.narrow(x, 1, 1, 2)
    tensor([[ 2,  3],
            [ 5,  6],
            [ 8,  9]])
Type:      builtin_function_or_method


RuntimeError: shape '[-1, 3750]' is invalid for input of size 2000000

In [17]:
train_dl = DataLoader(train_dataset, batch_size=1, num_workers=0)
# test_dl = DataLoader(test_dataset, batch_size=128, num_workers=0)

In [ ]:
model.eval()
for index, (xb, _) in enumerate(train_dl):
    with torch.no_grad():
        xb = xb.to(dev).view(-1,2000000)
        features = model.feature_aggregator(model.feature_extractor(xb))
        import pdb; pdb.set_trace()
        break;
        

> <ipython-input-19-5d7b14457b92>(7)<module>()
-> break;


(Pdb)  index


0


(Pdb)  xb


tensor([[ 0.0000,  0.0000,  0.0000,  ..., -0.0011, -0.0011, -0.0012]],
       device='cuda:0')


In [ ]:
import torch.optim as optim
import numpy as np
def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()
    print(loss.item())
    return loss.item(), len(xb)

def fit(epochs, model, loss_func, train_dl, valid_dl, opt=None):
    if not opt:
        opt = optim.SGD(model.parameters(), lr=0.0001)
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl:
            loss_batch(model, loss_func, xb, yb, opt)

        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl]
            )
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

        print(epoch, val_loss)

In [ ]:
loss_fn = nn.BCELoss()
fit(1, shrink, loss_fn, train_dl, test_dl)

In [ ]:
# def train(model, traindl, testdl):
#     model.train(True)
#     loss_fn = nn.BCELoss()
#     optimizer = optim.SGD(model.parameters(), lr=0.001)
#     for x, y in dl:
#         # generate random inputs and labels
#         x = x.to(dev)
#         y = y.to(dev)
#         # run forward pass
#         optimizer.zero_grad()
#         outputs = model(x)

#         # run backward pass
# #         y = y.to(outputs.device)
#         loss = loss_fn(outputs, y)
#         print(loss)
#         loss.backward
#         optimizer.step()

In [ ]:
train(shrink, testdl)